In [1]:
import os
import shutil
import sqlite3
from datetime import datetime, timedelta

def move_processed_files_to_deleted():
    try:
        # Constants
        completed_folder = "completed/"
        deleted_folder = "deleted/"

        # Connect to SQLite database
        conn = sqlite3.connect('conversion.db')
        cursor = conn.cursor()

        # Get current date and time
        current_datetime = datetime.now()

        # Calculate the threshold datetime (24 hours ago)
        threshold_datetime = current_datetime - timedelta(hours=24)

        # Get the list of processed files that are older than 24 hours
        cursor.execute('''SELECT local_file, source_file 
                          FROM ProcessedFile 
                          WHERE datetime(chunk_created_datetime) <= ? 
                          AND status = ?''', 
                       (threshold_datetime, 'Completed'))
        files_to_delete = cursor.fetchall()

        # Path to deleted folder
        deleted_folder_path = os.path.join(deleted_folder, current_datetime.strftime("%y%m%d"))

        # Create the deleted folder if it doesn't exist
        os.makedirs(deleted_folder_path, exist_ok=True)

        # Move processed files to deleted folder
        for file_name, source_file in files_to_delete:
            try:
                # Path to the completed file
                completed_file_path = os.path.join(completed_folder, source_file, "Chunks", file_name)
                
                # Move the file to the deleted folder
                destination_path = os.path.join(deleted_folder_path, file_name)
                shutil.move(completed_file_path, destination_path)
            except Exception as e:
                print(f"Error moving file {file_name} to deleted folder: {str(e)}")

        # If all files are moved successfully, update the status in the SourceFile table to "deleted"
        if len(files_to_delete) > 0:
            cursor.execute('''UPDATE SourceFile 
                              SET status=?, updated_datetime=? 
                              WHERE source_file IN (SELECT DISTINCT source_file FROM ProcessedFile 
                                                    WHERE datetime(chunk_created_datetime) <= ? 
                                                    AND status = ?)''', 
                           ('Deleted', current_datetime.strftime("%Y-%m-%d %H:%M:%S"), threshold_datetime, 'Completed'))
            conn.commit()

        # Close the database connection
        conn.close()
    except Exception as e:
        print(f"Error moving processed files to deleted folder: {str(e)}")

# Call the function to move processed files to the deleted folder
move_processed_files_to_deleted()


In [2]:
import os
import shutil
import sqlite3
from datetime import datetime, timedelta
import logging

def move_processed_files_to_deleted():
    try:
        # Constants
        completed_folder = "completed/"
        deleted_folder = "deleted/"

        # Connect to SQLite database
        conn = sqlite3.connect('conversion.db')
        cursor = conn.cursor()

        # Get current date and time
        current_datetime = datetime.now()

        # Calculate the threshold datetime (24 hours ago)
        threshold_datetime = current_datetime - timedelta(hours=24)

        # Get the list of processed files that are older than 24 hours
        cursor.execute('''SELECT local_file, source_file 
                          FROM ProcessedFile 
                          WHERE datetime(chunk_created_datetime) <= ? 
                          AND status = ?''', 
                       (threshold_datetime, 'Completed'))
        files_to_delete = cursor.fetchall()

        # Path to deleted folder
        deleted_folder_path = os.path.join(deleted_folder, current_datetime.strftime("%y%m%d"))

        # Create the deleted folder if it doesn't exist
        os.makedirs(deleted_folder_path, exist_ok=True)

        # Move processed files to deleted folder
        for file_name, source_file in files_to_delete:
            try:
                # Path to the completed file
                completed_file_path = os.path.join(completed_folder, source_file, "Chunks", file_name)
                
                # Move the file to the deleted folder
                destination_path = os.path.join(deleted_folder_path, file_name)
                shutil.move(completed_file_path, destination_path)
                logging.info(f"Moved file {file_name} to deleted folder.")
            except Exception as e:
                logging.error(f"Error moving file {file_name} to deleted folder: {str(e)}")

        # If all files are moved successfully, update the status in the SourceFile table to "deleted"
        if len(files_to_delete) > 0:
            cursor.execute('''UPDATE SourceFile 
                              SET status=?, updated_datetime=? 
                              WHERE source_file IN (SELECT DISTINCT source_file FROM ProcessedFile 
                                                    WHERE datetime(chunk_created_datetime) <= ? 
                                                    AND status = ?)''', 
                           ('Deleted', current_datetime.strftime("%Y-%m-%d %H:%M:%S"), threshold_datetime, 'Completed'))
            conn.commit()
            logging.info("Updated status in SourceFile table to 'Deleted'.")

        # Close the database connection
        conn.close()
        logging.info("Processed files moved to deleted folder successfully.")
    except Exception as e:
        logging.error(f"Error moving processed files to deleted folder: {str(e)}")

# Set up logging
log_file = 'logs/file_convert.log'
os.makedirs(os.path.dirname(log_file), exist_ok=True)
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Call the function to move processed files to the deleted folder
move_processed_files_to_deleted()


In [19]:
import os
import shutil
import sqlite3
from datetime import datetime, timedelta
import logging

def move_processed_files_to_deleted():
    try:
        # Constants
        completed_folder = "completed/"
        deleted_folder = "deleted/"

        # Connect to SQLite database
        conn = sqlite3.connect('conversion.db')
        cursor = conn.cursor()

        # Get current date and time
        current_datetime = datetime.now()

        # Calculate the threshold datetime (24 hours ago)
        threshold_datetime = current_datetime - timedelta(hours=24)

        # Get the list of processed files that are older than 24 hours
        cursor.execute('''SELECT local_file, source_file 
                          FROM ProcessedFile 
                          WHERE datetime(chunk_created_datetime) <= ? 
                          AND status = ?''', 
                       (threshold_datetime, 'Completed'))
        files_to_delete = cursor.fetchall()

        # If no files need to be deleted, exit the function
        if not files_to_delete:
            logging.info("No files to delete. Exiting the function.")
            return

        # Path to deleted folder
        deleted_folder_path = os.path.join(deleted_folder, current_datetime.strftime("%y%m%d"))

        # Create the deleted folder if it doesn't exist
        os.makedirs(deleted_folder_path, exist_ok=True)

        # Move processed files to deleted folder
        for file_name, source_file in files_to_delete:
            try:
                # Path to the completed file
                completed_file_path = os.path.join(completed_folder, source_file, "Chunks", file_name)
                
                # Move the file to the deleted folder
                destination_path = os.path.join(deleted_folder_path, file_name)
                shutil.move(completed_file_path, destination_path)
                logging.info(f"Moved file {file_name} to deleted folder.")
            except Exception as e:
                logging.error(f"Error moving file {file_name} to deleted folder: {str(e)}")

        # If all files are moved successfully, update the status in the SourceFile table to "deleted"
        cursor.execute('''UPDATE SourceFile 
                          SET status=?, updated_datetime=? 
                          WHERE source_file IN (SELECT DISTINCT source_file FROM ProcessedFile 
                                                WHERE datetime(chunk_created_datetime) <= ? 
                                                AND status = ?)''', 
                       ('Deleted', current_datetime.strftime("%Y-%m-%d %H:%M:%S"), threshold_datetime, 'Completed'))
        conn.commit()
        logging.info("Updated status in SourceFile table to 'Deleted'.")

        # Close the database connection
        conn.close()
        logging.info("Processed files moved to deleted folder successfully.")
    except Exception as e:
        logging.error(f"Error moving processed files to deleted folder: {str(e)}")

# Set up logging
log_file = 'logs/file_convert.log'
os.makedirs(os.path.dirname(log_file), exist_ok=True)
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Call the function to move processed files to the deleted folder
move_processed_files_to_deleted()


2024-04-18 12:01:42.416096
completed/
completed/
completed/
completed/
completed/
completed/
completed/
completed/
completed/
completed/
completed/
completed/
completed/
completed/
completed/
completed/
completed/
completed/
completed/
completed/
completed/
completed/
completed/


In [22]:
import os
import shutil
import sqlite3
from datetime import datetime, timedelta
import logging

def move_processed_folders_to_deleted():
    try:
        # Constants
        completed_folder = "completed/"
        deleted_folder = "deleted/"

        # Connect to SQLite database
        conn = sqlite3.connect('conversion.db')
        cursor = conn.cursor()

        # Get current date and time
        current_datetime = datetime.now()

        # Calculate the threshold datetime (24 hours ago)
        threshold_datetime = current_datetime - timedelta(days=1)

        # Get the list of processed folders that are older than 24 hours
        cursor.execute('''SELECT DISTINCT source_file 
                          FROM ProcessedFile 
                          WHERE DATE(chunk_created_datetime) <= ? 
                          AND status = ?''', 
                       (threshold_datetime.date(), 'Completed'))
        folders_to_delete = cursor.fetchall()

        # If no folders need to be deleted, exit the function
        if not folders_to_delete:
            logging.info("No folders to delete. Exiting the function.")
            return

        # Path to deleted folder
        deleted_folder_path = os.path.join(deleted_folder, threshold_datetime.strftime("%y%m%d"))

        # Create the deleted folder if it doesn't exist
        os.makedirs(deleted_folder_path, exist_ok=True)

        # Move processed folders to deleted folder
        for folder_name in folders_to_delete:
            try:
                # Path to the completed folder
                completed_folder_path = os.path.join(completed_folder, folder_name[0])
                print(completed_folder_path)
                
                # Move the folder to the deleted folder
                destination_path = os.path.join(deleted_folder_path, folder_name[0])
                shutil.move(completed_folder_path, destination_path)
                logging.info(f"Moved folder {folder_name[0]} to deleted folder.")
            except Exception as e:
                logging.error(f"Error moving folder {folder_name[0]} to deleted folder: {str(e)}")

        # If all folders are moved successfully, update the status in the SourceFile table to "deleted"
        cursor.execute('''UPDATE SourceFile 
                          SET status=?, updated_datetime=? 
                          WHERE source_file IN (SELECT DISTINCT source_file FROM ProcessedFile 
                                                WHERE DATE(chunk_created_datetime) <= ? 
                                                AND status = ?)''', 
                       ('Deleted', current_datetime.strftime("%Y-%m-%d %H:%M:%S"), threshold_datetime.date(), 'Completed'))
        conn.commit()
        logging.info("Updated status in SourceFile table to 'Deleted'.")

        # Close the database connection
        conn.close()
        logging.info("Processed folders moved to deleted folder successfully.")
    except Exception as e:
        logging.error(f"Error moving processed folders to deleted folder: {str(e)}")

# Set up logging
log_file = 'logs/folder_convert.log'
os.makedirs(os.path.dirname(log_file), exist_ok=True)
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Call the function to move processed folders to the deleted folder
move_processed_folders_to_deleted()


completed/054A40A4-9B9B-EB11-9DAE-E8393530AAAE.wav
completed/D046DB66-9C9B-EB11-9D8F-D89D67F437E7.wav
completed/8CA50AF3-9A9B-EB11-9DAE-E8393530AAAE.wav
completed/EC65ABC6-9F9B-EB11-B770-E8393530AA84.wav
completed/1247DB66-9C9B-EB11-9D8F-D89D67F437E7.wav


In [53]:
import os
import shutil
import sqlite3
from datetime import datetime, timedelta
import logging

def move_processed_folders_to_deleted():
    try:
        # Constants
        completed_folder = "completed/"
        deleted_folder = "deleted/"

        # Connect to SQLite database
        conn = sqlite3.connect('conversion.db')
        cursor = conn.cursor()

        # Get current date and time
        current_datetime = datetime.now()

        # Calculate the threshold datetime (24 hours ago)
        threshold_datetime = current_datetime - timedelta(days=1)

        # Get the list of processed folders that are older than 24 hours
        cursor.execute('''SELECT DISTINCT source_file, DATE(chunk_created_datetime) 
                          FROM ProcessedFile 
                          WHERE DATE(chunk_created_datetime) <= ? 
                          AND status = ?''', 
                       (threshold_datetime.date(), 'Completed'))
        folders_to_delete = cursor.fetchall()
        print(folders_to_delete)

        # If no folders need to be deleted, exit the function
        if not folders_to_delete:
            logging.info("No folders to delete. Exiting the function.")
            return

        # Path to deleted folder
        deleted_folder_path = os.path.join(deleted_folder)
        print(deleted_folder_path)

        # Create the deleted folder if it doesn't exist
        os.makedirs(deleted_folder_path, exist_ok=True)
        # Move processed folders to deleted folder
        for source_file, download_datetime in folders_to_delete:
            try:
                # Path to the completed folder
                print("aaa")
                date_object = datetime.datetime.strptime(download_datetime, '%Y-%m-%d')
                print(date_object)
                formatted_date = date_object.strftime("%y%m%d")
                print(formatted_date)

                completed_folder_path = os.path.join(completed_folder, download_datetime.strftime("%yy%mm%dd"))
                print(completed_folder_path)
                # Move the folder to the deleted folder
                destination_path = os.path.join(deleted_folder_path, source_file)
                shutil.move(completed_folder_path, destination_path)
                logging.info(f"Moved folder {source_file} to deleted folder.")
            except Exception as e:
                logging.error(f"Error moving folder {source_file} to deleted folder: {str(e)}")

        # If all folders are moved successfully, update the status in the SourceFile table to "deleted"
        cursor.execute('''UPDATE SourceFile 
                          SET status=?, updated_datetime=? 
                          WHERE source_file IN (SELECT DISTINCT source_file FROM ProcessedFile 
                                                WHERE DATE(chunk_created_datetime) <= ? 
                                                AND status = ?)''', 
                       ('Deleted', current_datetime.strftime("%Y-%m-%d %H:%M:%S"), threshold_datetime.date(), 'Completed'))
        conn.commit()
        logging.info("Updated status in SourceFile table to 'Deleted'.")

        # Close the database connection
        conn.close()
        logging.info("Processed folders moved to deleted folder successfully.")
    except Exception as e:
        logging.error(f"Error moving processed folders to deleted folder: {str(e)}")

# Set up logging
log_file = 'logs/folder_convert.log'
os.makedirs(os.path.dirname(log_file), exist_ok=True)
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Call the function to move processed folders to the deleted folder
move_processed_folders_to_deleted()


[('054A40A4-9B9B-EB11-9DAE-E8393530AAAE.wav', '2024-04-16'), ('D046DB66-9C9B-EB11-9D8F-D89D67F437E7.wav', '2024-04-16'), ('8CA50AF3-9A9B-EB11-9DAE-E8393530AAAE.wav', '2024-04-16'), ('EC65ABC6-9F9B-EB11-B770-E8393530AA84.wav', '2024-04-16'), ('1247DB66-9C9B-EB11-9D8F-D89D67F437E7.wav', '2024-04-16')]
deleted/
aaa
aaa
aaa
aaa
aaa


In [62]:
import os
import shutil
import sqlite3
from datetime import datetime, timedelta
import logging

def move_processed_folders_to_deleted():
    try:
        # Constants
        completed_folder = "completed/"
        deleted_folder = "deleted/"

        # Connect to SQLite database
        conn = sqlite3.connect('conversion.db')
        cursor = conn.cursor()

        # Get current date and time
        current_datetime = datetime.now()

        # Calculate the threshold datetime (24 hours ago)
        threshold_datetime = current_datetime - timedelta(days=1)

        # Get the list of processed folders that are older than 24 hours
        cursor.execute('''SELECT DISTINCT source_file, DATE(chunk_created_datetime) 
                          FROM ProcessedFile 
                          WHERE DATE(chunk_created_datetime) <= ? 
                          AND status = ?''', 
                       (threshold_datetime.date(), 'Completed'))
        folders_to_delete = cursor.fetchall()

        # If no folders need to be deleted, exit the function
        if not folders_to_delete:
            logging.info("No folders to delete. Exiting the function.")
            return

        # Path to deleted folder
        deleted_folder_path = os.path.join(deleted_folder)

        # Create the deleted folder if it doesn't exist
        os.makedirs(deleted_folder_path, exist_ok=True)
        # Move processed folders to deleted folder
        for source_file, download_datetime in folders_to_delete:
                # Path to the completed folder
                date_object = datetime.strptime(download_datetime, '%Y-%m-%d')  
                completed_folder_path = os.path.join(completed_folder, date_object.strftime("%y%m%d"))  
                # Move the folder to the deleted folder
                destination_path = os.path.join(deleted_folder_path)
                shutil.move(completed_folder_path, destination_path)
                logging.info(f"Moved folder {completed_folder_path} to deleted folder.")

        # If all folders are moved successfully, update the status in the SourceFile table to "deleted"
        cursor.execute('''UPDATE SourceFile 
                          SET status=?, updated_datetime=? 
                          WHERE source_file IN (SELECT DISTINCT source_file FROM ProcessedFile 
                                                WHERE DATE(chunk_created_datetime) <= ? 
                                                AND status = ?)''', 
                       ('Deleted', current_datetime.strftime("%Y-%m-%d %H:%M:%S"), threshold_datetime.date(), 'Completed'))
        conn.commit()
        logging.info("Updated status in SourceFile table to 'Deleted'.")

        # Close the database connection
        conn.close()
        logging.info("Processed folders moved to deleted folder successfully.")
    except Exception as e:
        logging.error(f"Error moving processed folders to deleted folder: {str(e)}")

# Set up logging
log_file = 'logs/folder_convert.log'
os.makedirs(os.path.dirname(log_file), exist_ok=True)
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Call the function to move processed folders to the deleted folder
move_processed_folders_to_deleted()
